In [1]:
pip install flask pandas sklearn seaborn matplotlib


  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
      
      More information is available at
      https://github.com/scikit-learn/sklearn-

In [9]:
from flask import Flask, render_template, request, redirect, url_for
from threading import Thread
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt
import io
import base64

app = Flask(__name__)

# Global variables to store results and thread status
results = {}
analysis_thread = None
thread_running = False

def analyze_diseases():
    global results, thread_running

    # Set thread status
    thread_running = True

    # Load the dataset
    data = pd.read_csv(r'Blood_samples_dataset_with_patient_index_2.csv')

    # Drop irrelevant columns (Patient_ID, etc.)
    X = data.drop(columns=['Patient_ID', 'Disease'])
    y = data['Disease']

    # Encode the target variable (Disease)
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)

    # Scale features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Split the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42)

    # Create and train the Random Forest classifier
    clf = RandomForestClassifier(n_estimators=100, random_state=42)
    clf.fit(X_train, y_train)

    # Make predictions on the test data
    y_pred = clf.predict(X_test)

    # Evaluate the model's performance
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred, target_names=label_encoder.classes_, output_dict=True)

    # Analyze Disease Prevalence
    predicted_diseases = label_encoder.inverse_transform(y_pred)
    disease_counts = pd.Series(predicted_diseases).value_counts()

    # Plot Disease Prevalence
    plt.figure(figsize=(10, 6))
    sns.barplot(x=disease_counts.index, y=disease_counts.values, palette='viridis')
    plt.title('Disease Prevalence in the City')
    plt.xlabel('Disease')
    plt.ylabel('Number of Patients')
    plt.xticks(rotation=90)
    
    # Save plot to a string buffer
    buffer = io.BytesIO()
    plt.savefig(buffer, format='png')
    buffer.seek(0)
    plot_data = base64.b64encode(buffer.getvalue()).decode('utf-8')
    buffer.close()

    # Output the most common disease
    most_common_disease = disease_counts.idxmax()

    # Provide health recommendations based on the most prevalent disease
    if most_common_disease == 'Diabetes':
        recommendation = "Focus on diet improvements (reducing sugar), increasing physical activity, and regular glucose screening programs to manage and prevent diabetes."
    elif most_common_disease == 'Anemia':
        recommendation = "Focus on iron-rich diets (leafy greens, red meat) and providing iron supplements, especially for vulnerable groups like children and women."
    elif most_common_disease == 'Thalassemia':
        recommendation = "Enhance screening for thalassemia, provide genetic counseling, and ensure treatment facilities for managing the condition."
    elif most_common_disease == 'Heart Disease':
        recommendation = "Promote heart-healthy diets, regular physical activity, and smoking cessation programs to reduce the risk of heart disease."
    else:
        recommendation = "General health improvements, including promoting exercise, balanced diets, and regular check-ups, should be implemented."

    # Store the results globally
    results = {
        'accuracy': accuracy * 100,
        'classification_report': report,
        'disease_counts': disease_counts,
        'plot_data': plot_data,
        'most_common_disease': most_common_disease,
        'recommendation': recommendation
    }

    # Set thread status
    thread_running = False

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/analyze', methods=['POST'])
def analyze():
    global analysis_thread, thread_running

    if not thread_running:
        # Start a new thread to run the analysis
        analysis_thread = Thread(target=analyze_diseases)
        analysis_thread.start()
        return redirect(url_for('results'))
    else:
        return redirect(url_for('index'))

@app.route('/results')
def results():
    if thread_running:
        return render_template('index.html', message="Analysis is still running. Please wait...")
    
    if not results:
        return render_template('index.html', message="No results available yet.")
    
    return render_template('results.html', 
                           accuracy=results['accuracy'], 
                           classification_report=results['classification_report'],
                           disease_counts=results['disease_counts'].to_dict(),
                           plot_data=results['plot_data'],
                           most_common_disease=results['most_common_disease'],
                           recommendation=results['recommendation'])

# Run the Flask app
if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


 * Restarting with stat


SystemExit: 1